In [1]:
import cv2


In [2]:
import mediapipe as mp
import numpy as np
from scipy.signal import lfilter
import matplotlib.pyplot as plt

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [4]:
hands = mp_hands.Hands(max_num_hands = 2, static_image_mode = False,min_detection_confidence = 0.5)

In [5]:
# storing all the coordinates at different timesteps in a list
list_x = []
list_y = []
list_z = []
list_avg_x = []
list_avg_y = []
list_avg_z = []


In [6]:
# color and thickness of the lines
color = (255,0,0)
thickness = 2
radius = 2
# threshold after which we should start drawing
threshold = 8

# threshold after which we should take base depth
base_threshold = 6



# moving average buffer
k = 5



In [7]:
# canvas on which we are drawing
canvas = np.zeros([512,512,1],dtype=np.uint8)
display = canvas.copy()
cursor = canvas.copy()

In [8]:
toggle = 1

     
    

In [9]:
base_depth = 0

In [10]:
lift_threshold = -10
touch_threshold = -25

In [11]:
cap = cv2.VideoCapture(0)
buffer_count = 0
delay_count = 0
while cap.isOpened():
    success, image = cap.read()
    
    toggle = 0
    if not success:
        print("video capture unsuccessful")
        break
    
    
     
    image = cv2.cvtColor(cv2.flip(image,1), cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    
    
    
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    if results.multi_hand_landmarks:
        
        
        
        buffer_count = buffer_count+1
        hand_landmarks = results.multi_hand_landmarks[0]
        
        # add the coordinates to the list
        list_x.append(int(hand_landmarks.landmark[8].x*512))
        list_y.append(int(hand_landmarks.landmark[8].y*512))
        list_z.append(int(hand_landmarks.landmark[8].z*512))
        
        
        
        
        
        if(buffer_count == k):
            avg_x = sum(list_x[-k:])/k
            avg_y = sum(list_y[-k:])/k
            avg_z = sum(list_z[-k:])/k
            
            list_avg_x.append(int(avg_x))
            list_avg_y.append(int(avg_y))
            list_avg_z.append(int(avg_z))
            buffer_count = 0
            
           
            
            if(len(list_avg_z) == base_threshold):
                base_depth = list_avg_z[-1]
                
            if(list_avg_z[-1] - base_depth > lift_threshold):
               toggle = 0
            if(list_avg_z[-1] - base_depth < touch_threshold):
                toggle = 1
            
            if(len(list_avg_x) > threshold and toggle == 1):
                # get the current coordinates as endpoint
                end_point = (list_avg_x[-1], list_avg_y[-1])
                # get the previous coordinates as the start point
                start_point = (list_avg_x[-2], list_avg_y[-2])
                # draw a line joining these points
                cv2.line(canvas,start_point, end_point,color, thickness)
                display = canvas.copy()
            elif(len(list_avg_x) > threshold and toggle == 0):
                
                # get the current coordinates as endpoint
                end_point = (list_avg_x[-1], list_avg_y[-1])
                cursor = canvas.copy()
                cv2.circle(cursor,end_point,radius,color,thickness)
                display = cursor.copy()
                
                
                
       
        mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
    
  
    
    # show the canvas and the webcam image   
    cv2.imshow("Detected Hands", image)
    cv2.imshow("Result", display)
    
    
    
    # exit when the user presses 's's
    if cv2.waitKey(5) & 0xFF == ord('s'):
        break
hands.close()
cv2.destroyAllWindows()
cap.release()






In [12]:
print(list_z)

[-75, -16, -76, -55, -4, -38, -32, -35, -33, -7, -22, -17, -10, -22, -22, -10, -15, -19, -6, -13, -19, -39, -12, -9, -18, -15, -13, -22, -11, -7, -20, -10, -13, -13, -15, -11, -5, -5, -8, -16, -7, -30, -10, -15, -18, -6, -11, -9, -16, -4, -21, -23, -20, -10, -25, -11, -7, -12, -6, -13, -16, -11, -10, -5, -21, -19, -15, -17, -5, -12, -11, -17, -11, -17, -19, -16, -15, -18, -20, -20, -30, -37, -49, -49, -53, -52, -44, -54, -47, -57, -64, -65, -76, -70, -66, -64, -65, -66, -69, -66, -69, -65, -64, -78, -77, -73, -79, -75, -74, -89, -79, -82, -80, -79, -81, -84, -82, -99, -89, -90, -101, -106, -97, -100, -100, -97, -101, -102, -94, -102, -108, -98, -106, -119, -117, -101, -109, -102, -99, -93, -94, -93, -93, -94, -100, -99, -92, -100, -91, -97, -102, -109, -103, -98, -95, -100, -104, -94, -107, -107, -109, -109, -110, -119, -98, -115, -97, -104, -110, -115, -100, -115, -114, -119, -109, -104, -107, -108, -111, -97, -101, -95, -95, -97, -90, -83, -96, -77, -74, -79, -80, -78, -51, -53, -52,